In [12]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS

In [13]:
df = pd.read_csv(
    "/Users/alexandralugova/Documents/GitHub/MH-old-workers/data/datasets/data_clean.csv"
).rename(columns={"work_horizon_change": "whc"})

/var/folders/ck/3npmnx597nb730qdy5w_6kp00000gn/T/ipykernel_25149/4066616281.py:1: DtypeWarning: Columns (5,22,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [14]:
df_waves = []
for w in df.wave.unique():
    if w > 1 and w < 8:
        df_wave0 = df[df.wave == w]
        df_wave0["post"] = 0

        df_wave1 = df[df.wave == w]
        df_wave1["post"] = 1

        df_waves.append(df_wave0)
        df_waves.append(df_wave1)

    elif w == 1:
        df_wave0 = df[df.wave == w]
        df_wave0["post"] = 0
        df_waves.append(df_wave0)
    else:
        df_wave1 = df[df.wave == w]
        df_wave1["post"] = 1
        df_waves.append(df_wave1)

df = pd.concat(df_waves, sort=False, axis=0).reset_index(drop=True)

/var/folders/ck/3npmnx597nb730qdy5w_6kp00000gn/T/ipykernel_25149/2343401198.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wave0["post"] = 0
/var/folders/ck/3npmnx597nb730qdy5w_6kp00000gn/T/ipykernel_25149/2343401198.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wave0["post"] = 0
/var/folders/ck/3npmnx597nb730qdy5w_6kp00000gn/T/ipykernel_25149/2343401198.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [15]:
df["treated"] = df["whc"] > 0
df["treated"] = df["treated"].astype("int")

df["treated_post"] = df["treated"] * df["post"]

In [16]:
df["treated0105"] = (df["whc"] > 0) & (df["whc"] <= 0.5)
df["treated0105_post"] = df["treated0105"] * df["post"]

df["treated0612"] = (df["whc"] > 0.5) & (df["whc"] <= 1)
df["treated0612_post"] = df["treated0612"] * df["post"]

df["treated13plus"] = df["whc"] > 1
df["treated13plus_post"] = df["treated13plus"] * df["post"]

In [17]:
df["block"] = 0

conditions = [
    (df["wave"] == 1) | ((df["wave"] == 2) & (df["post"] == 1)),
    ((df["wave"] == 2) & (df["post"] == 0)) | ((df["wave"] == 4) & (df["post"] == 1)),
    ((df["wave"] == 4) & (df["post"] == 0)) | ((df["wave"] == 5) & (df["post"] == 1)),
    ((df["wave"] == 5) & (df["post"] == 0)) | ((df["wave"] == 6) & (df["post"] == 1)),
    ((df["wave"] == 6) & (df["post"] == 0)) | ((df["wave"] == 7) & (df["post"] == 1)),
    ((df["wave"] == 7) & (df["post"] == 0)) | (df["wave"] == 8),
]

values = [1, 2, 3, 4, 5, 6]

df["block"] = np.select(conditions, values)

In [18]:
df.groupby("block").mergeid.nunique()

block
1    5681
2    9310
3    6932
4    6787
5    6617
6    3260
Name: mergeid, dtype: int64

In [22]:
df.groupby("block")["mergeid"].value_counts().to_frame().value_counts()

count
2        25635
1         7552
4         3174
3         2226
Name: count, dtype: int64

In [9]:
df["mergeid_count"] = df.groupby("block")["mergeid"].transform("count")

df = df[df["mergeid_count"] == 2].copy()

In [11]:
df

,mergeid,wave,country,dn002_,dn003_,dn006_,dn042_,ch001_,ep005_,ep009_,...,treated,treated_post,treated0105,treated0105_post,treated0612,treated0612_post,treated13plus,treated13plus_post,block,mergeid_count


In [10]:
df.groupby("block").mergeid.nunique()

Series([], Name: mergeid, dtype: int64)

In [9]:
# Linear specification
linear_model = sm.OLS.from_formula(
    "eurod ~ post + whc + whc:post + C(block) + C(job_demands):C(block)", data=df_models
)
linear_results = linear_model.fit()

# Binary treatment specification
binary_model = sm.Logit.from_formula(
    "eurod ~ post + treated + treated:post + C(block) + C(job_demands):C(block)",
    data=df_models,
)
binary_results = binary_model.fit()

# Non-linear specification
nonlinear_model = sm.OLS.from_formula(
    "eurod ~ post + treated0105 + treated0105:post + treated0612 + treated0612:post +  treated13plus + treated13plus:post + C(block) + C(job_demands):C(block)",
    data=df_models,
)
nonlinear_results = nonlinear_model.fit()

ValueError: endog must be in the unit interval.

In [ ]:
fixed_effects = pd.get_dummies(df_models["gender"], drop_first=True)

In [ ]:
df_models = pd.concat([df_models, fixed_effects], axis=1)

In [ ]:
df_models["Male"] = df_models["Male"].astype("int")

In [ ]:
model = sm.OLS(
    df_models["eurod"],
    sm.add_constant(
        df_models[
            [
                "treated",
                "post",
                "treated_post",
                "Male",
                "job_passive",
                "job_active",
                "job_high_strain",
                "job_low_strain",
                "job_poor_recognition",
                "job_poor_prospects",
                "job_insecurity",
            ]
        ]
    ),
)

# Fit the model
result = model.fit()

# Print the results
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  eurod   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                     636.6
Date:                Fri, 10 Nov 2023   Prob (F-statistic):               0.00
Time:                        11:12:27   Log-Likelihood:            -1.5667e+05
No. Observations:               78196   AIC:                         3.134e+05
Df Residuals:                   78184   BIC:                         3.135e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                -1.077e+09 